# Electricity market - Model 1

## Situation
Simple electricity market model. No network.

Uses solver engine 'cbc' or 'appsi_highs', to compare the results as a way of testing the HiGHS solver.

Can uses data files 1 to 3.

In [1]:
# Import dependencies

import pyomo.environ as pyo
import pandas as pd
import os.path
import json

In [2]:
# Get data

DataFilename = os.path.join('.', 'market-data-1.json')
with open(DataFilename, 'r') as f:
    Data = json.load(f)

In [3]:
# Declarations

Model = pyo.ConcreteModel(name = 'Electricity market Model 1 - ' + Data['Name'])

Model.Demand = pyo.Param(within = pyo.NonNegativeReals, initialize = Data['Demand'])

Model.VarInitial = pyo.Param(within = pyo.NonNegativeReals, initialize = Data['VarInitial'])
Model.VarLBounds = pyo.Param(within = pyo.NonNegativeReals, initialize = Data['VarLBounds'])
Model.VarUBounds = pyo.Param(within = pyo.NonNegativeReals, initialize = Data['VarUBounds'])
Model.Engine = pyo.Param(within = pyo.Any, initialize = Data['Engine'])
Model.TimeLimit = pyo.Param(within = pyo.NonNegativeReals, initialize = Data['TimeLimit'])

Generators = Data['Generators']
Model.Generators = pyo.Set(initialize = list(Generators.keys()))                 # Pyomo Set rather than Python set

Model.VarCost = pyo.Param(Model.Generators, within = pyo.NonNegativeReals, mutable = True)
Model.GMin = pyo.Param(Model.Generators, within = pyo.NonNegativeReals, mutable = True)
Model.GMax = pyo.Param(Model.Generators, within = pyo.Reals, mutable = True)

for s in Model.Generators:    
    Model.VarCost[s] = Generators[s]['VarCost']
    Model.GMin[s] = Generators[s]['GMin']
    Model.GMax[s] = Generators[s]['GMax']

In [4]:
# Define model

Model.Dispatch = pyo.Var(Model.Generators, domain = pyo.NonNegativeReals, initialize = Model.VarInitial, bounds = (Model.VarLBounds, Model.VarUBounds))

def rule_demand(Model):
    return sum(Model.Dispatch[s] for s in Model.Generators) == Model.Demand   # Total generation must meet demand
Model.MeetDemand = pyo.Constraint(rule = rule_demand)

def rule_mustrun(Model, S):
    return Model.Dispatch[S] >= Model.GMin[S]   # Minimum dispatch for each generator. May include must run minimum
Model.MustRun = pyo.Constraint(Model.Generators, rule = rule_mustrun)

def rule_capacity(Model, S):
    return Model.Dispatch[S] <= Model.GMax[S]   # Maximum dispatch for each generator
Model.MaxCapacity = pyo.Constraint(Model.Generators, rule = rule_capacity)

def rule_Obj(Model):
    return sum(Model.VarCost[s] * Model.Dispatch[s] for s in Model.Generators)   # Total cost of dispatched generation
Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.minimize)

In [5]:
# Solve model

Solver = pyo.SolverFactory(pyo.value(Model.Engine))
Model.dual = pyo.Suffix(direction = pyo.Suffix.IMPORT)
Results = Solver.solve(Model, load_solutions = False, tee = False)

ApplicationError: Solver <class 'pyomo.contrib.appsi.base.SolverFactoryClass.register.<locals>.decorator.<locals>.LegacySolver'> is not available (NotFound).

In [ ]:
# Process results

WriteSolution = False
Optimal = False
LimitStop = False
Condition = Results.solver.termination_condition

if Condition == pyo.TerminationCondition.optimal:
    Optimal = True
if Condition == pyo.TerminationCondition.maxTimeLimit or Condition == pyo.TerminationCondition.maxIterations:
    LimitStop = True
if Optimal or LimitStop:
    try:
        WriteSolution = True
        Model.solutions.load_from(Results)                                     # Defer loading results until now, in case there is no solution to load
        SolverData = Results.Problem._list
        SolutionLB = SolverData[0].lower_bound
        SolutionUB = SolverData[0].upper_bound
    except:
        WriteSolution = False

In [ ]:
# Write output

print(Model.name, '\n')
print('Status:', Results.solver.termination_condition)
print('Solver:', pyo.value(Model.Engine), '\n')

if LimitStop:                                                                  # Indicate how close we are to a solution
    print('Objective bounds')
    print('----------------')
    if SolutionLB is None:
        print('Lower:      None')
    else:
        print(f'Lower: {SolutionLB:9,.2f}')
    if SolutionUB is None:
        print('Upper:      None\n')
    else:
        print(f'Upper: {SolutionUB:9,.2f}\n')
if WriteSolution:
    print(f'Total cost = ${Model.Obj():,.2f}\n')
    pd.options.display.float_format = "{:,.2f}".format
    GenResults = pd.DataFrame()
    for s in Model.Generators:
        GenResults.loc[s, 'Dispatch'] = pyo.value(Model.Dispatch[s])
    display(GenResults)

    ConstraintStatus = pd.DataFrame(columns=['lSlack', 'uSlack', 'Dual'])
    for c in Model.component_objects(pyo.Constraint, active = True):
        for index in c:  # Allow for indexed contraints, like rule_capacity
            ConstraintStatus.loc[c[index].name] = [c[index].lslack(), c[index].uslack(), Model.dual[c[index]]]
    display(ConstraintStatus)
else:
    print('No solution loaded\n')
    print('Model:')
    Model.pprint()